In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
from hilbert import decode, encode
from tqdm import tqdm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_parquet("data/F_BTCUSDT_1m.parquet")
tc = ["Open",
"High",
"Low",
"Close",
"Volume"]
for i in tc:
    df[i]=df[i].astype("float")

In [221]:
data = np.array(range(32*32))
locs = decode(data, 2, 5)
hilbert_list = []
target_list = []
target_list_real = []


for i in tqdm(range(32*32,len(df),10)):
    temp_f = df.iloc[i-32*32:i]
    #temp_t_5 = df.iloc[i:i+5]
    #temp_t_10 = df.iloc[i+5:i+10]
    temp_t_15 = df.iloc[i:i+15]
    #temp_t_20 = df.iloc[i+15:i+20]
    
    if len(temp_t_15)<5:
        target_list.append(
        np.array([0,0,0,0,0,0,0,0])
    )
        continue
    
    lv = temp_f.Close.iloc[-1]
    o1 = np.zeros((5,32,32))
    vals = temp_t_15[["Open","High","Low","Close","Volume"]].values
    target_list_real.append(temp_t_15[["Open","High","Low","Close"]].values)
    vals[:,:4]=(vals[:,:4]-vals[:,2].min())/(vals[:,1].max()-vals[:,2].min())
    vals[:,4]=(vals[:,4]-vals[:,4].min())/(vals[:,4].max()-vals[:,4].min())
    #for j in range(len(data)):
    #    o1[:,locs[j][0],locs[j][1]] = vals[j]

#    hilbert_list.append(o1)
    target_list.append(vals)
    
    """target_list.append(
        np.array(
            [
                (temp_t_5.Low.min()/lv)-1,(temp_t_5.High.max()/lv)-1,
                (temp_t_10.Low.min()/lv)-1,(temp_t_10.High.max()/lv)-1,
                (temp_t_15.Low.min()/lv)-1,(temp_t_15.High.max()/lv)-1,
                (temp_t_20.Low.min()/lv)-1,(temp_t_20.High.max()/lv)-1
            ]
        )
    )"""
    

 47%|███████████████▉                  | 74374/158298 [00:46<00:54, 1549.92it/s]/tmp/ipykernel_4140/2372953184.py:25: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:26: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:25: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:26: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:25: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:26: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:25: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:26: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:25: RuntimeWarning:

invalid value encountered in true_divide

/tmp/ipykernel_4140/2372953184.py:26: RuntimeWa

In [222]:
target_list_real[-1]=target_list_real[-2]

In [223]:
target_list = [i.flatten() for i in target_list]

In [224]:
target_list[-1]=target_list[-2]

In [225]:
target_list = [i.reshape(1,*i.shape) for i in target_list]

In [226]:
tla = np.concatenate(target_list,axis=0)

In [227]:
tla.shape

(158298, 75)

In [45]:
tla = tla.reshape(tla.shape[0],tla.shape[1]*tla.shape[2])

In [46]:
tla.shape

(158298, 75)

In [8]:
h1=temp_t_15["High"].iloc[0]
h2=temp_t_15["High"].iloc[7]
h3=temp_t_15["High"].iloc[14]
ha = (((h2/h1)-1)+((h3/h2)-1))/2
l1=temp_t_15["Low"].iloc[0]
l2=temp_t_15["Low"].iloc[7]
l3=temp_t_15["Low"].iloc[14]
la = (((l2/l1)-1)+((l3/l2)-1))/2
la,ha

(-0.0020184114768714845, -0.0018619093889628013)

In [5]:
target_df = pd.DataFrame(target_list,columns=["Low_5","High_5","Low_10","High_10","Low_15","High_15","Low_20","High_20"])

In [59]:
target_df.iloc[-1]=target_df.iloc[-2]

In [18]:
target_df["Low_slope"] = ((-((target_df.Low_10/target_df.Low_5)-1)) + (-((target_df.Low_15/target_df.Low_10)-1)) + (-((target_df.Low_20/target_df.Low_15)-1)))/3
target_df["High_slope"] = ((((target_df.High_10/target_df.High_5)-1)) + (((target_df.High_15/target_df.High_10)-1)) + (((target_df.High_20/target_df.High_15)-1)))/3


In [42]:
tdf2 = (tdf2-tdf2.values.mean())/tdf2.values.std()

In [44]:
target_df = tdf1.join(tdf2)

In [4]:
hlt = np.array(hilbert_list)

In [5]:
ohlc = hlt[:,:4,:,:]
vol = hlt[:,-1:,:,:]

In [6]:
ohlc.mean(),ohlc.std(),vol.mean(),vol.std()

(28001.938870730588, 17834.47169281324, 248.00923831004107, 426.5438396373735)

In [5]:
ohlc = (ohlc-ohlc.mean())/ohlc.std()
vol = (vol-vol.mean())/vol.std()
hltn = np.concatenate((ohlc,vol),axis=1)
hltn.shape

In [6]:
hltn = np.concatenate((ohlc,vol),axis=1)
hltn.shape

(158298, 5, 32, 32)

In [61]:
#hlt = (hlt-hlt.mean())/hlt.std()
div = int(len(target_list)*0.9)
#for i,j in tqdm(enumerate(hltn)):
#    if i<div:
#        np.save('imgs/train_4layer_w_volume/%07d.npy'%i,j)
#    else:
#        np.save('imgs/test_4layer_w_volume/%07d.npy'%i,j) 
target_df["label"] = target_df.index.to_list()
target_df["label"] = target_df.label.apply(lambda x:"%07d.npy"%x)
target_df.iloc[:div].to_parquet("imgs/train_4layer_w_volume/target_line_3.parquet")
target_df.iloc[div:].to_parquet("imgs/test_4layer_w_volume/target_line_3.parquet")

In [228]:
tla = np.nan_to_num(tla)

In [229]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit_predict(tla)


In [38]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit_predict(target_df.iloc[:,:-2].values)


In [55]:
np.unique(kmeans,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19], dtype=int32),
 array([7961, 8348, 9492, 9549, 8612, 7651, 6564, 8356, 9797, 5913, 8325,
        8082, 8413, 5892, 7545, 7637, 7875, 6932, 6296, 9058]))

In [230]:
dfc = pd.DataFrame(tla)
dfc["label"] = kmeans

In [231]:
dsc = dfc.groupby("label").describe()

In [232]:
col_list = []
for i,j in enumerate(dsc.columns):
    if j[0]%5==0:
        val="Open"
    if j[0]%5==1:
        val="High"
    if j[0]%5==2:
        val="Low"
    if j[0]%5==3:
        val="Close"
    if j[0]%5==4:
        val="Volume"
    col_list.append(val+"_"+str((j[0]//5)+1)+"_"+j[1]) 

In [233]:
dsc.columns=col_list

In [101]:
dsc.to_parquet("20_cluster_desc.parquet")

In [41]:
target_df["labels"]=kmeans

In [30]:
tla

array([[8.35472e+03, 8.35595e+03, 8.35375e+03, ..., 8.32005e+03,
        8.32485e+03, 1.77890e+01],
       [8.32041e+03, 8.32497e+03, 8.31904e+03, ..., 8.31423e+03,
        8.32200e+03, 2.34280e+01],
       [8.34404e+03, 8.34854e+03, 8.34167e+03, ..., 8.33611e+03,
        8.34186e+03, 5.41500e+00],
       ...,
       [2.03035e+04, 2.03150e+04, 2.03010e+04, ..., 2.03332e+04,
        2.03362e+04, 2.18129e+02],
       [2.03253e+04, 2.03253e+04, 2.03021e+04, ..., 2.03299e+04,
        2.03440e+04, 5.95273e+02],
       [2.03253e+04, 2.03253e+04, 2.03021e+04, ..., 2.03299e+04,
        2.03440e+04, 5.95273e+02]])

In [84]:
dsc = pd.read_parquet("20_cluster_desc.parquet")

In [153]:
np.array([ta[[j+"_"+str(i)+"_mean" for i in range(1,16)]].to_list() for j in ["Open","High","Low","Close"]]).transpose()

array([[1.00000014, 1.00050549, 0.99949736, 0.99999538],
       [0.99999523, 1.0005439 , 0.99945056, 0.99998758],
       [0.99998758, 1.00049886, 0.99948234, 0.99999244],
       [0.99999257, 1.00049624, 0.99949488, 0.999985  ],
       [0.99998534, 1.00047494, 0.99949247, 0.99997734],
       [0.99997688, 1.00046857, 0.99949614, 0.99997232],
       [0.99997212, 1.00053876, 0.9994444 , 0.99999348],
       [0.99999371, 1.00049089, 0.99946937, 0.99996677],
       [0.99996704, 1.00046886, 0.99943501, 0.99993834],
       [0.99993799, 1.00054706, 0.9992615 , 0.99987931],
       [0.99988027, 1.00103297, 0.99867024, 0.99990395],
       [0.99990301, 1.00071938, 0.99915947, 0.99995427],
       [0.99995398, 1.00057096, 0.99931628, 0.99993812],
       [0.99993745, 1.00050631, 0.99934598, 0.9999205 ],
       [0.99992095, 1.00047462, 0.99936369, 0.99990515]])

In [209]:
target_list_real[46]

array([[8312.87, 8314.38, 8308.51, 8310.  ],
       [8306.86, 8310.91, 8306.86, 8308.02],
       [8310.24, 8315.68, 8306.86, 8312.32],
       [8312.22, 8312.48, 8309.09, 8309.09],
       [8311.62, 8311.62, 8308.28, 8309.47],
       [8309.39, 8309.76, 8306.86, 8306.86],
       [8308.92, 8308.92, 8303.1 , 8304.64],
       [8304.64, 8308.  , 8276.37, 8279.02],
       [8276.35, 8290.53, 8275.81, 8283.14],
       [8281.51, 8285.56, 8279.73, 8282.78],
       [8282.53, 8284.75, 8280.38, 8281.05],
       [8281.21, 8287.36, 8279.32, 8286.7 ],
       [8287.57, 8292.07, 8286.79, 8292.07],
       [8290.96, 8294.93, 8289.08, 8292.68],
       [8289.08, 8291.02, 8285.23, 8287.82]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,label
46,0.000032,0.000214,-0.000492,-0.000313,0.144588,-0.000691,-0.000203,-0.000691,-0.000551,0.040120,-0.000284,0.000371,-0.000691,-0.000034,0.287935,-0.000046,-0.000014,-0.000422,-0.000422,0.078183,-0.000118,-0.000118,-0.000520,-0.000377,0.084020,-0.000386,-0.000342,-0.000691,-0.000691,0.000000,-0.000443,-0.000443,-0.001143,-0.000958,0.285921,-0.000958,-0.000553,-0.004358,-0.004040,1.000000,-0.004361,-0.002655,-0.004426,-0.003544,0.530057,-0.003740,-0.003253,-0.003954,-0.003587,0.106642,-0.003617,-0.003350,-0.003876,-0.003795,0.174886,-0.003776,-0.003036,-0.004004,-0.003116,0.064143,-0.003011,-0.002470,-0.003105,-0.002470,0.036749,-0.002603,-0.002126,-0.002829,-0.002396,0.391380,-0.002829,-0.002596,-0.003293,-0.002981,0.143128,4
65,-0.000019,0.000096,-0.000167,-0.000167,0.000000,-0.000213,-0.000055,-0.000465,-0.000055,0.441366,-0.000092,-0.000092,-0.001601,-0.001423,0.222120,-0.001410,-0.000952,-0.001440,-0.000953,0.045046,-0.001186,-0.000681,-0.001299,-0.001064,0.184876,-0.001136,-0.001136,-0.002030,-0.002030,0.253855,-0.001994,-0.001994,-0.002512,-0.002512,0.380585,-0.002359,-0.002359,-0.003233,-0.002506,0.923825,-0.002542,-0.000984,-0.002542,-0.001552,0.687946,-0.001536,-0.000854,-0.001628,-0.000996,0.080208,-0.001034,-0.000541,-0.001034,-0.000542,0.217876,-0.000542,0.000490,-0.000542,-0.000223,1.000000,0.000027,0.000061,-0.000312,-0.000312,0.154063,-0.000330,-0.000292,-0.000542,-0.000542,0.096655,-0.000487,-0.000436,-0.000707,-0.000671,0.157832,4
99,0.000000,0.000361,-0.000755,-0.000659,0.389485,-0.000659,-0.000414,-0.001170,-0.000653,0.000000,-0.000659,0.000213,-0.000864,0.000071,0.042634,0.000086,0.001673,0.000086,0.001316,0.458240,0.001341,0.001469,0.000647,0.000847,0.299809,0.000676,0.000676,-0.000166,-0.000064,0.499584,-0.000147,0.000697,-0.000309,-0.000219,0.052741,-0.000229,0.000088,-0.000831,-0.000229,1.000000,-0.000009,0.001255,-0.000009,0.000468,0.119083,0.000649,0.000649,-0.000093,0.000479,0.050370,0.000484,0.000484,-0.000170,0.000367,0.077365,0.000280,0.001516,0.000211,0.001507,0.505199,0.001507,0.001714,0.001073,0.001197,0.101447,0.001168,0.001307,0.000768,0.001179,0.190375,0.001194,0.001285,0.000714,0.000925,0.267241,4
101,0.000037,0.000683,0.000037,0.000683,0.000000,0.000666,0.000688,0.000275,0.000676,0.053994,0.000322,0.000610,-0.000043,0.000225,0.051962,-0.000043,0.000289,-0.000074,-0.000044,0.034078,-0.000044,-0.000044,-0.000135,-0.000044,0.006959,-0.000044,0.001479,-0.000135,0.001111,0.106009,0.001111,0.001407,0.000932,0.001407,0.041848,0.001407,0.008285,0.001407,0.002964,1.000000,0.002884,0.003046,0.002014,0.002544,0.123471,0.002496,0.002799,0.001707,0.002759,0.076670,0.002758,0.002758,0.001756,0.001852,0.031339,0.001922,0.002258,0.001732,0.002255,0.116736,0.002255,0.002255,0.001965,0.002108,0.023622,0.002108,0.002799,0.002108,0.002799,0.154837,0.002799,0.003344,0.002793,0.003306,0.117661,4
102,-0.000001,-0.000001,-0.001000,-0.000904,0.020314,-0.000835,-0.000500,-0.001025,-0.000503,0.245086,-0.000503,-0.000503,-0.000792,-0.000650,0.000000,-0.000650,0.000040,-0.000650,0.000040,0.345371,0.000040,0.000583,0.000034,0.000545,0.247521,0.000403,0.000403,-0.000204,0.000029,0.108631,0.000242,0.001112,0.000242,0.000976,0.099575,0.001149,0.002590,0.001117,0.001643,1.000000,0.001643,0.001731,0.000391,0.000863,0.354359,0.000825,0.001446,0.000679,0.001176,0.110382,0.001272,0.001978,0.000699,0.000902,0.114335,0.000902,0.002007,0.000902,0.001661,0.239833,0.001851,0.002052,0.001836,0.001951,0.216563,0.001951,0.001953,0.000591,0.000627,0.285755,0.000557,0.000557,0.000186,0.000298,0.105881,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [234]:
dsc

,Open_1_count,Open_1_mean,Open_1_std,Open_1_min,Open_1_25%,Open_1_50%,Open_1_75%,Open_1_max,High_1_count,High_1_mean,High_1_std,High_1_min,High_1_25%,High_1_50%,High_1_75%,High_1_max,Low_1_count,Low_1_mean,Low_1_std,Low_1_min,Low_1_25%,Low_1_50%,Low_1_75%,Low_1_max,Close_1_count,Close_1_mean,Close_1_std,Close_1_min,Close_1_25%,Close_1_50%,Close_1_75%,Close_1_max,Volume_1_count,Volume_1_mean,Volume_1_std,Volume_1_min,Volume_1_25%,Volume_1_50%,Volume_1_75%,Volume_1_max,Open_2_count,Open_2_mean,Open_2_std,Open_2_min,Open_2_25%,Open_2_50%,Open_2_75%,Open_2_max,High_2_count,High_2_mean,High_2_std,High_2_min,High_2_25%,High_2_50%,High_2_75%,High_2_max,Low_2_count,Low_2_mean,Low_2_std,Low_2_min,Low_2_25%,Low_2_50%,Low_2_75%,Low_2_max,Close_2_count,Close_2_mean,Close_2_std,Close_2_min,Close_2_25%,Close_2_50%,Close_2_75%,Close_2_max,Volume_2_count,Volume_2_mean,Volume_2_std,Volume_2_min,Volume_2_25%,Volume_2_50%,Volume_2_75%,Volume_2_max,Open_3_count,Open_3_mean,Open_3_std,Open_3_min,Open_3_25%,Open_3_50%,Open_3_75%,Open_3_max,High_3_count,High_3_mean,High_3_std,High_3_min,High_3_25%,High_3_50%,High_3_75%,High_3_max,Low_3_count,Low_3_mean,Low_3_std,Low_3_min,Low_3_25%,Low_3_50%,Low_3_75%,Low_3_max,Close_3_count,Close_3_mean,Close_3_std,Close_3_min,Close_3_25%,Close_3_50%,Close_3_75%,Close_3_max,Volume_3_count,Volume_3_mean,Volume_3_std,Volume_3_min,Volume_3_25%,Volume_3_50%,Volume_3_75%,Volume_3_max,Open_4_count,Open_4_mean,Open_4_std,Open_4_min,Open_4_25%,Open_4_50%,Open_4_75%,Open_4_max,High_4_count,High_4_mean,High_4_std,High_4_min,High_4_25%,High_4_50%,High_4_75%,High_4_max,Low_4_count,Low_4_mean,Low_4_std,Low_4_min,Low_4_25%,Low_4_50%,Low_4_75%,Low_4_max,Close_4_count,Close_4_mean,Close_4_std,Close_4_min,Close_4_25%,Close_4_50%,Close_4_75%,Close_4_max,Volume_4_count,Volume_4_mean,Volume_4_std,Volume_4_min,Volume_4_25%,Volume_4_50%,Volume_4_75%,Volume_4_max,Open_5_count,Open_5_mean,Open_5_std,Open_5_min,Open_5_25%,Open_5_50%,Open_5_75%,Open_5_max,High_5_count,High_5_mean,High_5_std,High_5_min,High_5_25%,High_5_50%,High_5_75%,High_5_max,Low_5_count,Low_5_mean,Low_5_std,Low_5_min,Low_5_25%,Low_5_50%,Low_5_75%,Low_5_max,Close_5_count,Close_5_mean,Close_5_std,Close_5_min,Close_5_25%,Close_5_50%,Close_5_75%,Close_5_max,Volume_5_count,Volume_5_mean,Volume_5_std,Volume_5_min,Volume_5_25%,Volume_5_50%,Volume_5_75%,Volume_5_max,Open_6_count,Open_6_mean,Open_6_std,Open_6_min,Open_6_25%,Open_6_50%,Open_6_75%,Open_6_max,High_6_count,High_6_mean,High_6_std,High_6_min,High_6_25%,High_6_50%,High_6_75%,High_6_max,Low_6_count,Low_6_mean,Low_6_std,Low_6_min,Low_6_25%,Low_6_50%,Low_6_75%,Low_6_max,Close_6_count,Close_6_mean,Close_6_std,Close_6_min,Close_6_25%,Close_6_50%,Close_6_75%,Close_6_max,Volume_6_count,Volume_6_mean,Volume_6_std,Volume_6_min,Volume_6_25%,Volume_6_50%,Volume_6_75%,Volume_6_max,Open_7_count,Open_7_mean,Open_7_std,Open_7_min,Open_7_25%,Open_7_50%,Open_7_75%,Open_7_max,High_7_count,High_7_mean,...,Close_9_75%,Close_9_max,Volume_9_count,Volume_9_mean,Volume_9_std,Volume_9_min,Volume_9_25%,Volume_9_50%,Volume_9_75%,Volume_9_max,Open_10_count,Open_10_mean,Open_10_std,Open_10_min,Open_10_25%,Open_10_50%,Open_10_75%,Open_10_max,High_10_count,High_10_mean,High_10_std,High_10_min,High_10_25%,High_10_50%,High_10_75%,High_10_max,Low_10_count,Low_10_mean,Low_10_std,Low_10_min,Low_10_25%,Low_10_50%,Low_10_75%,Low_10_max,Close_10_count,Close_10_mean,Close_10_std,Close_10_min,Close_10_25%,Close_10_50%,Close_10_75%,Close_10_max,Volume_10_count,Volume_10_mean,Volume_10_std,Volume_10_min,Volume_10_25%,Volume_10_50%,Volume_10_75%,Volume_10_max,Open_11_count,Open_11_mean,Open_11_std,Open_11_min,Open_11_25%,Open_11_50%,Open_11_75%,Open_11_max,High_11_count,High_11_mean,High_11_std,High_11_min,High_11_25%,High_11_50%,High_11_75%,High_11_max,Low_11_count,Low_11_mean,Low_11_std,Low_11_min,Low_11_25%,Low_11_50%,Low_11_75%,Low_11_max,Close_11_count,Close_11_mean,Close_11_std,Close_11_min,Close_11_25%,Close_11_50%,Close_11_75%,Close_11_max,Volume_11_count

In [253]:
idcs = list(dfc[dfc.label==0].index )

In [254]:
idcs

[3,
 36,
 42,
 70,
 91,
 134,
 135,
 140,
 154,
 177,
 232,
 262,
 266,
 298,
 301,
 304,
 343,
 345,
 346,
 362,
 384,
 441,
 459,
 469,
 517,
 520,
 595,
 617,
 621,
 625,
 673,
 693,
 729,
 741,
 795,
 802,
 811,
 838,
 842,
 860,
 864,
 872,
 897,
 972,
 1003,
 1033,
 1040,
 1063,
 1090,
 1123,
 1162,
 1176,
 1178,
 1197,
 1215,
 1222,
 1351,
 1368,
 1371,
 1377,
 1381,
 1393,
 1477,
 1482,
 1493,
 1510,
 1570,
 1591,
 1618,
 1621,
 1663,
 1677,
 1687,
 1706,
 1721,
 1726,
 1732,
 1735,
 1808,
 1812,
 1825,
 1836,
 1882,
 1918,
 1937,
 1958,
 1973,
 1986,
 1989,
 2002,
 2017,
 2025,
 2073,
 2083,
 2088,
 2097,
 2105,
 2108,
 2124,
 2136,
 2185,
 2231,
 2237,
 2249,
 2264,
 2271,
 2278,
 2282,
 2320,
 2353,
 2364,
 2376,
 2378,
 2410,
 2415,
 2438,
 2444,
 2462,
 2510,
 2512,
 2517,
 2566,
 2569,
 2592,
 2601,
 2620,
 2622,
 2631,
 2632,
 2661,
 2665,
 2680,
 2702,
 2721,
 2751,
 2755,
 2805,
 2812,
 2835,
 2866,
 2880,
 2890,
 2914,
 2927,
 2936,
 2956,
 2964,
 2978,
 2982,
 2994,


In [266]:
np.concatenate(target_list_real).max()

69198.7

In [280]:
pd.DataFrame([[target_list_real[i][0,0]/target_list_real[i].min(),target_list_real[i].max()/target_list_real[i][0,0]] for i in idcs]).describe()

,0,1
count,9497.000000,9497.000000
mean,1.001192,1.005122
std,0.001389,0.005510
min,1.000000,1.000000
25%,1.000355,1.002174
50%,1.000836,1.003585
75%,1.001563,1.006035
max,1.024757,1.091107


In [279]:
label_i = 2
ta = dsc.loc[label_i]
idcs = list(dfc[dfc.label==label_i].index )
to_plot = pd.DataFrame(np.array([ta[[j+"_"+str(i)+"_mean" for i in range(1,16)]].to_list() for j in ["Open","High","Low","Close"]]).transpose(),columns=["Open","High","Low","Close"])
to_plot = to_plot
to_plot["time"]=range(1,16)
fig1 = go.Figure(data=go.Ohlc(x=to_plot['time'],
                             open=to_plot['Open'],
                             high=to_plot['High'],
                             low=to_plot['Low'],
                             close=to_plot['Close']))
  
# show the figure
fig1.update(layout_xaxis_rangeslider_visible=False)
fig1.show()
for i in range(10):
    print(idcs[i*len(idcs)//10])
    real_vals = target_list_real[idcs[i*len(idcs)//10]]
    print(real_vals[0,0]/real_vals.min(),real_vals.max()/real_vals[0,0])
    to_plot_real = pd.DataFrame(real_vals,columns=["Open","High","Low","Close"])
    to_plot_real["time"]=range(1,16)
    fig = go.Figure(data=go.Ohlc(x=to_plot_real['time'],
                                 open=to_plot_real['Open'],
                                 high=to_plot_real['High'],
                                 low=to_plot_real['Low'],
                                 close=to_plot_real['Close']))

    # show the figure
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.show()


14
1.0005059666901268 1.0124773958049968


15563
1.001103505665642 1.0039054470709148


31343
1.0008832499152858 1.0040182932244028


47271
1.0000929406859365 1.0012788814050644


62226
1.0000005237933112 1.003063665472244


78327
1.0026142624668228 1.002739115093088


95624
1.0001057498131751 1.006702523811426


112315
1.0013126464191746 1.0098489558987707


129056
1.001262039860267 1.00301499430277


144251
1.0007975912743514 1.0034567950429358


In [270]:
20150/20060,39780/39620

(1.0044865403788634, 1.0040383644623927)

In [250]:
import plotly.graph_objects as go
  
# import python pandas package
import pandas as pd
  
# read the stock price dataset
  
# use go.OHLC function and pass the date, open,
# high, low and close price of the function


In [ ]:
from matplotlib.finance import candlestick2_ohlc
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as datetime
import numpy as np

quotes = np.array(...)

fig, ax = plt.subplots()
candlestick2_ohlc(ax,quotes['open'],quotes['high'],quotes['low'],quotes['close'],width=0.6)

xdate = [datetime.datetime.fromtimestamp(i) for i in quotes['time']]

ax.xaxis.set_major_locator(ticker.MaxNLocator(6))

def mydate(x,pos):
    try:
        return xdate[int(x)]
    except IndexError:
        return ''

ax.xaxis.set_major_formatter(ticker.FuncFormatter(mydate))

fig.autofmt_xdate()
fig.tight_layout()

plt.show()